In [1]:
""" Makes tax units from the ASEC.

Based on Sam Portnow's code at 
https://users.nber.org/~taxsim/to-taxsim/cps/cps-portnow/TaxSimRScriptForDan.R
"""
import numpy as np
import pandas as pd

# personal exemptions

pexemp = pd.DataFrame({
  'year': [2018],
  'pexemp': [0]})


ipum = pd.read_csv('~/UBICenter/covid_ui/asec_2019_ipums.csv.gz')
# ipum = pd.read_csv('~/MaxGhenis/datarepo/asec_2019_ipums.csv.gz')
# set to lower case
ipum.columns = ipum.columns.str.lower()

# Set missing income items to zero so that non-filers etc will get zeroes.
VARS_MISSING_ZERO = [
    'eitcred', 'fedretir', 'fedtax', 'statetax', 'adjginc', 'taxinc',
    'fedtaxac', 'fica', 'stataxac', 'incdivid', 'incint', 'incrent',
    'incother', 'incasist', 'incss', 'incwelfr', 'incwkcom', 'incvet',
    'incchild', 'incunemp', 'inceduc', 'gotveduc', 'gotvothe', 'gotvpens',
    'gotvsurv', 'incssi', 'incwage', 'incbus', 'incfarm', 'incsurv',
    'incdisab', 'incretir', 'inccapg']


# these are the missing codes
MISSING_CODES = [9999, 99999, 999999, 9999999,
                 -9999, -99999, -999999, -9999999,
                 9997, 99997, 999997, 9999997]

for var in VARS_MISSING_ZERO:
    ipum.loc[ipum[var].isna() | ipum[var].isin(MISSING_CODES), var] = 0

assert (ipum[VARS_MISSING_ZERO].isin(MISSING_CODES)).sum().sum() == 0
assert (ipum[VARS_MISSING_ZERO].isna()).sum().sum() == 0

In [2]:
# set 0's to NA for location
COLS_ZERO_TO_NA = ['momloc', 'poploc', 'sploc']
for col in COLS_ZERO_TO_NA:
    ipum.loc[ipum[col] == 0, col] = np.nan


# year before tax returns
ipum['x2'] = ipum.year - 1

# set x3 to fips code
ipum['x3'] = ipum.statefip

# Marital status will be sum of spouse's x4 values
ipum['x4'] = 1

# https://cps.ipums.org/cps-action/variables/RELATE#codes_section
RELATE_COHABITORS_NOT_MARRIED_CODE = 1114
ipum.loc[ipum.relate == RELATE_COHABITORS_NOT_MARRIED_CODE, 'sploc'] = np.nan

# x6 is age of unmarried people and primary married people (sploc > pernum).
# x24 is age of everyone else.
ipum['unmarried_primary_married'] = (
    ipum.sploc.isna() | ((ipum.sploc > 0) & (ipum.sploc > ipum.pernum)))
ipum['x6'] = np.where(ipum.unmarried_primary_married, ipum.age, 0)
ipum['x24'] = ipum.age - ipum.x6


# primary wage or spouse wage
ipum['incwagebusfarm'] = ipum[['incwage', 'incbus', 'incfarm']].sum(axis=1)
ipum['x7'] = np.where(ipum.unmarried_primary_married, ipum.incwagebusfarm, 0)
ipum['x8'] = ipum.incwagebusfarm - ipum.x7

# Add some income variables.
ipum['x9'] = ipum.incdivid
ipum['x10'] = ipum[['incrent', 'incother']].sum(axis=1)
ipum['x11'] = ipum.incretir
ipum['x12'] = ipum.incss
ipum['x27'] = ipum.incint
ipum['x28'] = 0

# Other income.
ipum['x13'] = ipum[['incwelfr', 'incwkcom', 'incvet', 'incsurv', 'incdisab',
                     'incchild', 'inceduc', 'incssi', 'incasist']].sum(axis=1)

ipum['x14'] = ipum.incrent
ipum['x15'] = 0


# /* use Census imputation of itemized deductions where available.*/
# first have to join the exemption table
pexemp.rename(columns={'year': 'x2'}, inplace=True)
ipum = ipum.merge(pexemp, on='x2')

# adjusted gross - taxes + exemptions
ipum['x16'] = (ipum.adjginc - 
    ipum[['pexemp', 'statetax', 'taxinc']].sum(axis=1))
# no values less than 0
ipum['x16'] = np.where(ipum.x16 < 0, 0, ipum.x16)

ipum['x17'] = 0
ipum['x18'] = ipum.incunemp
ipum['x19'] = 0
ipum['x20'] = 0
ipum['x21'] = 0

# Assume inccapg is long term (caploss is no longer in IPUMS CPS)
ipum['x22'] = ipum.inccapg


# Here we output a record for each person, so that tax units can be formed 
# later by summing over person records. The taxunit id is the minimum of
# the pernum or sploc, so spouses will get the same id. For children
# it is the minimum of the momloc or poploc. Other relatives are made
# dependent on the household head (which may be incorrect) and non-relatives
# are separate tax units. 
# */

RELATE_HEAD_OF_HOUSEHOLD_CODE = 101
ipum['hnum'] = np.where(ipum.relate == RELATE_HEAD_OF_HOUSEHOLD_CODE,
                        ipum.pernum, np.nan)

# People with income above the personal exemption must file separately.
ipum['claimed_income'] = ipum[['x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13',
                               'x22']].sum(axis=1)
ipum['notself'] = np.where(ipum.claimed_income <= ipum.pexemp, 1, 0)

# If someone is a dependent of their spouse, set dependent pointer to 0.
ipum.loc[~ipum.sploc.isna() & (ipum.depstat > 0) & (ipum.depstat == ipum.sploc),
         'depstat'] = 0

ipum['is_dep'] = ipum.depstat > 0

# Dependent children must be dependents with a parent who is below age 18,
# or below age 24 if in school.
ipum['depchild'] = np.where(
    ipum.is_dep &
    (~ipum.momloc.isna() | ~ipum.poploc.isna()) &
    ((ipum.age < 18) | ((ipum.age < 24) & (ipum.schlcoll > 0))),
    1, 0)

# Dependent relatives are dependents who are not dependent children.
ipum['deprel'] = np.where(ipum.is_dep & (ipum.depchild == 0), 1, 0)
# Define dependent children under age 13/17/18.
for i in [13, 17, 18]:
    ipum['dep' + str(i)] = np.where(ipum.is_dep & (ipum.age < i), 1, 0)

ipum.groupby(['is_dep', 'depchild', 'deprel']).size()

is_dep  depchild  deprel
False   0         0         126480
True    0         1           2517
        1         0          51104
dtype: int64

In [3]:
# set dependents and taxpayers
deps = ipum[ipum.is_dep].copy(deep=True)

# Define identifier as 100 * serial (household) + tax unit sub-identifier
deps['x1'] = (100 * deps.serial +
              np.where(deps.depchild, np.fmin(deps.momloc, deps.poploc),
                       deps.hnum))

deps['x4'] = np.nan
deps['x5'] = 1
deps['x6'] = 0
deps['x19'] = np.nan
deps['x23'] = np.nan
deps['x24'] = 0

In [4]:
txpyrs = ipum[~ipum.is_dep].copy(deep=True)
txpyrs['x1'] = 100 * txpyrs.serial + np.fmin(txpyrs.pernum, txpyrs.sploc)
txpyrs['x5'] = 0
txpyrs['x23'] = np.nan


# set whats not x1, x2, or x5 in deps to NA
vars = ['x' + str(i) for i in [3, 4, 27, 28] + list(range(6, 23))]
deps[vars] = np.nan

# put them back together
ipum2 = pd.concat([txpyrs, deps])


# sum value over tax #
ipum2['n'] = 1
ipum2.rename({'dep17': 'x25', 'dep18': 'x26'}, axis=1, inplace=True)

concat_sum = ipum2.groupby(['x2', 'x1'])[
    ['n'] + ['x' + str(i) for i in list(range(3, 29))]].sum()
concat_sum.x3 /= concat_sum.n
# x6 and x24 should be max not sum, and n is no longer necessary.
concat_sum.drop(['x6', 'x24', 'n'], axis=1, inplace=True)

concat_max = ipum2.groupby(['x2', 'x1'])[['x6', 'x24']].max()
concat_min = ipum2.groupby(['x2', 'x1'])[['serial', 'pernum']].min()
concat_min.columns = ['x29', 'x30']

concat = concat_sum.join(concat_max).join(concat_min).reset_index()

concat = concat[(concat.x19 >= 0) & (concat.x4) > 0]

concat = concat[['x' + str(i) for i in list(range(1, 31))]]

concat.columns = ['taxsimid', 'year', 'state', 'mstat', 'depx', 'page',
                  'pwages', 'swages', 'dividends', 'otherprop', 'pensions',
                  'gssi', 'transfers', 'rentpaid', 'proptax', 'otheritem',
                  'childcare', 'ui', 'depchild', 'mortgage', 'stcg', 'ltcg',
                  'dep13', 'sage', 'dep17', 'dep18', 'intrec', 'nonprop',
                  'serial', 'pernum']

ids = concat[['taxsimid', 'serial', 'pernum']]

In [77]:
crosswalk = ipum2[['serial', 'pernum', 'x1', 'spmfamunit']].copy(deep=True)
crosswalk.rename({'x1': 'taxsimid'}, axis=1, inplace=True)

In [6]:
concat.to_csv('2018_taxsim_units.csv', index=False)

In [11]:
concat.depx.unique()

array([ 0,  1,  9,  2,  3,  4,  5,  7,  6,  8, 10, 11])

Define crosswalk from tax unit to SPM unit.

**NOTE: Some tax units map to multiple SPM units.**

In [90]:
ipum.shape

(180101, 92)

In [92]:
ipum2.index

Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            180070, 180073, 180080, 180081, 180088, 180089, 180090, 180091,
            180097, 180098],
           dtype='int64', length=180101)

In [89]:
ipum.cpsidp.nunique()

118109

In [87]:
ipum2.columns

Index(['year', 'serial', 'month', 'cpsid', 'asecflag', 'asecwth', 'statefip',
       'pernum', 'cpsidp', 'asecwt', 'relate', 'age', 'sex', 'race', 'momloc',
       'poploc', 'sploc', 'schlcoll', 'wksunem1', 'incwage', 'incbus',
       'incfarm', 'incss', 'incwelfr', 'incretir', 'incssi', 'incint',
       'incunemp', 'incwkcom', 'incvet', 'incsurv', 'incdisab', 'incdivid',
       'incrent', 'inceduc', 'incchild', 'incasist', 'incother', 'inccapg',
       'gotveduc', 'gotvothe', 'gotvpens', 'gotvsurv', 'adjginc', 'eitcred',
       'fedretir', 'fedtax', 'fedtaxac', 'fica', 'depstat', 'statetax',
       'stataxac', 'taxinc', 'spmwt', 'spmtotres', 'spmthresh', 'spmfamunit',
       'x2', 'x3', 'x4', 'unmarried_primary_married', 'x6', 'x24',
       'incwagebusfarm', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x27', 'x28',
       'x13', 'x14', 'x15', 'pexemp', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21',
       'x22', 'hnum', 'claimed_income', 'notself', 'is_dep', 'depchild',
       'deprel', 'dep13'

In [79]:
crosswalk.taxsimid.nunique()

86394

In [80]:
crosswalk[['taxsimid', 'spmfamunit']].drop_duplicates().shape[0]

88310

Each person gets the share of the tax change in accord with their UI.

In [ ]:
tu_spm_crosswalk = 

## Convert to taxcalc

Rename and process per https://github.com/PSLmodels/Tax-Calculator/blob/master/taxcalc/validation/taxsim27/prepare_taxcalc_input.py

In [12]:
TAXSIM_TAXCALC_COLS = {
    'taxsimid': 'RECID',
    'year': 'FLPDYR',
    'mstat': 'mstat',
    'page': 'age_head',
    'sage': 'age_spouse',
    'depx': 'num_deps',
    'dep13': 'f2441',
    'dep17': 'n24',
    'dep18': 'num_eitc_qualified_kids',
    'pwages': 'e00200p',
    'swages': 'e00200s',
    'dividends': 'e00650',
    'intrec': 'e00300',
    'stcg': 'p22250',
    'ltcg': 'p23250',
    'otherprop': 'e02000',
    'nonprop': 'e00800',
    'pensions': 'e01700',
    'gssi': 'e02400',
    'ui': 'e02300',
    'proptax': 'e18500',
    'otheritem': 'e18400',
    'childcare': 'e32800',
    'mortgage': 'e19200',
}

In [13]:
cps_tc = concat.rename(columns=TAXSIM_TAXCALC_COLS)

### Preprocess

In [34]:
cps_tc['MARS'] = np.where(cps_tc.mstat == 1,
                          np.where(cps_tc.num_deps > 0, 4, 1), 2)
cps_tc['EIC'] = np.minimum(cps_tc.num_eitc_qualified_kids, 3)
num_taxpayers = np.where(cps_tc.MARS == 2, 2, 1)
cps_tc['XTOT'] = num_taxpayers + cps_tc.num_deps
cps_tc['e00200'] = cps_tc['e00200p'] + cps_tc['e00200s']
cps_tc['e00600'] = cps_tc['e00650']
cps_tc['e01500'] = cps_tc['e01700']

## Run tax analysis

In [36]:
import taxcalc as tc
import microdf as mdf

In [58]:
concat[(concat.mstat == 1) & (concat.swages != 0)].page

13274    0.0
47924    0.0
77911    0.0
81577    0.0
Name: page, dtype: float64

In [60]:
single_spouses = cps_tc[(cps_tc.e00200s != 0) & (cps_tc.MARS == 1)]
cps_tc[cps_tc.RECID.isin(single_spouses.RECID)]

,RECID,FLPDYR,state,mstat,num_deps,age_head,e00200p,e00200s,e00650,e02000,...,e00300,e00800,serial,pernum,MARS,EIC,XTOT,e00200,e00600,e01500
47924,5458701.0,2018,47.0,1.0,0,0.0,0.0,16000.0,0.0,0.0,...,0.0,0.0,54587,4,1,0,1,16000.0,0.0,0.0
81577,9000601.0,2018,6.0,1.0,0,0.0,0.0,6000.0,0.0,0.0,...,0.0,0.0,90006,5,1,0,1,6000.0,0.0,0.0


In [70]:
# TODO: Figure out why this is happening.
# Manually fix this for now.
single_spouses = cps_tc[(cps_tc.e00200s != 0) &
                        (cps_tc.MARS !=2)].copy(deep=True)
print(str(single_spouses.shape[0]),
      ' records are single but have nonzero spouse wages')
single_spouses.e00200p = single_spouses.e00200s
single_spouses.e00200s = 0
single_spouses.age_head = single_spouses.age_spouse
single_spouses.age_spouse = 0
# Recombine
cps_tc = pd.concat([cps_tc[~cps_tc.RECID.isin(single_spouses.RECID)],
                   single_spouses])

2  records are single but have nonzero spouse wages


In [71]:
calcdf = mdf.calc_df(records=tc.Records(cps_tc))

RECID
401         4790.157719
601            0.000000
701         2763.843475
801            0.000000
802         2763.843475
               ...     
9463302    10880.084454
5458701     4091.791300
9000601     1204.600893
1516201    -4834.439590
8678301    18768.067513
Name: tax, Length: 86022, dtype: float64

In [66]:
cps_tc.columns

Index(['RECID', 'FLPDYR', 'state', 'mstat', 'num_deps', 'age_head', 'e00200p',
       'e00200s', 'e00650', 'e02000', 'e01700', 'e02400', 'transfers',
       'rentpaid', 'e18500', 'e18400', 'e32800', 'e02300', 'depchild',
       'e19200', 'p22250', 'p23250', 'f2441', 'age_spouse', 'n24',
       'num_eitc_qualified_kids', 'e00300', 'e00800', 'serial', 'pernum',
       'MARS', 'EIC', 'XTOT', 'e00200', 'e00600', 'e01500'],
      dtype='object')

In [50]:
crosswalk[crosswalk.serial == 54587]

,serial,pernum,taxsimid
99400,54587,2,5458702.0
99402,54587,4,5458701.0
99399,54587,1,5458702.0
99401,54587,3,5458702.0


In [49]:
crosswalk[crosswalk.taxsimid.isin([5458701, 9000601])]

,serial,pernum,taxsimid
99402,54587,4,5458701.0
170344,90006,5,9000601.0


In [45]:
cps_tc[(cps_tc.e00200s != 0) & cps_tc.MARS.isin([1, 5])]

,RECID,FLPDYR,state,mstat,num_deps,age_head,e00200p,e00200s,e00650,e02000,...,e00300,e00800,serial,pernum,MARS,EIC,XTOT,e00200,e00600,e01500
47924,5458701.0,2018,47.0,1.0,0,0.0,0.0,16000.0,0.0,0.0,...,0.0,0.0,54587,4,1,0,1,16000.0,0.0,0.0
81577,9000601.0,2018,6.0,1.0,0,0.0,0.0,6000.0,0.0,0.0,...,0.0,0.0,90006,5,1,0,1,6000.0,0.0,0.0


In [15]:
cps_tc

,RECID,FLPDYR,state,mstat,num_deps,age_head,e00200p,e00200s,e00650,e02000,...,p22250,p23250,f2441,age_spouse,n24,num_eitc_qualified_kids,e00300,e00800,serial,pernum
0,401.0,2018,23.0,1.0,0,21.0,18000.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0.0,0.0,4,1
1,601.0,2018,23.0,1.0,0,85.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0.0,0.0,6,1
2,701.0,2018,23.0,1.0,0,61.0,12000.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0.0,0.0,7,1
3,801.0,2018,23.0,1.0,0,73.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0.0,0.0,8,1
4,802.0,2018,23.0,1.0,0,37.0,12000.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0.0,0.0,8,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86389,9463101.0,2018,5.0,2.0,4,30.0,11000.0,3000.0,0.0,0.0,...,0.0,0.0,0.0,33,4,4,0.0,0.0,94631,1
86390,9463201.0,2018,15.0,2.0,0,80.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,73,0,0,10.0,0.0,94632,1
86391,9463203.0,2018,7.5,2.0,2,57.0,28000.0,30000.0,0.0,0.0,...,0.0,0.0,0.0,45,2,2,2.0,0.0,94632,3
86392,9463301.0,2018,15.0,1.0,0,57.0,15000.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0.0,0.0,94633,1


In [9]:
concat.columns

Index(['taxsimid', 'year', 'state', 'mstat', 'depx', 'page', 'pwages',
       'swages', 'dividends', 'otherprop', 'pensions', 'gssi', 'transfers',
       'rentpaid', 'proptax', 'otheritem', 'childcare', 'ui', 'depchild',
       'mortgage', 'stcg', 'ltcg', 'dep13', 'sage', 'dep17', 'dep18', 'intrec',
       'nonprop', 'serial', 'pernum'],
      dtype='object')

In [8]:
concat.shape

(86022, 30)